In [1]:
import netCDF4 as nc4
import keras
import xarray,numpy as np
import math
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from keras.datasets import mnist
from keras.models import Model
from keras.layers import Input, add
from keras.layers.core import  Dense, Dropout, Activation, Flatten, Reshape
from keras import regularizers
from keras.regularizers import l2
from keras.layers.convolutional import Conv2D, MaxPooling2D, UpSampling2D, ZeroPadding2D
from keras.utils import np_utils
from tensorflow.keras import layers

# Read data

In [2]:
file_path =['C:/Users/myxll/OneDrive - The University of Melbourne/capstone/data/OFAM_2017/ocean_eta_t/ocean_eta_t_2000_01.nc']
file_path.append('C:/Users/myxll/OneDrive - The University of Melbourne/capstone/data/OFAM_2017/ocean_eta_t/ocean_eta_t_2000_02.nc')
# file_path.append('C:/Users/myxll/OneDrive - The University of Melbourne/capstone/data/OFAM_2017/ocean_eta_t/ocean_eta_t_2000_03.nc')
# file_path.append('C:/Users/myxll/OneDrive - The University of Melbourne/capstone/data/OFAM_2017/ocean_eta_t/ocean_eta_t_2000_04.nc')
# file_path.append('C:/Users/myxll/OneDrive - The University of Melbourne/capstone/data/OFAM_2017/ocean_eta_t/ocean_eta_t_2000_05.nc')
# file_path.append('C:/Users/myxll/OneDrive - The University of Melbourne/capstone/data/OFAM_2017/ocean_eta_t/ocean_eta_t_2000_06.nc')
ds = nc4.MFDataset(file_path)
eta_t = ds.variables['eta_t'][:]
eta_t_numpy = np.array(eta_t)
eta_t_numpy = np.nan_to_num(eta_t_numpy)
eta_t_numpy.shape

(60, 1500, 3600)

# fixed missing values with 0

In [3]:
def fix_missing_value(input_data):
    for i in range(0,len(input_data)):
        arr = input_data[i]
        arr[arr == -32768] = 0

# Normalize the input

In [4]:
def normalize_matrix(matrix):
    norm = np.linalg.norm(matrix)
    matrix = matrix/norm  # normalized matrix
    return matrix


# Split array into 128*128

In [5]:
import math
lat = 256
long = 256
def split_array(input_data):
    pad_rows = math.ceil(3600/lat)*lat - 3600
    pad_cols = math.ceil(1500/long)*long - 1500
#     print(pad_rows)
#     print(pad_cols)
    
    input_data=np.pad(input_data,((0,pad_cols),(0,pad_rows)), 'constant',constant_values=(0,0)) # padding with zeros
    
    l = np.array_split(input_data,len(input_data)/lat,axis=0)
    input_data_split = []
    for i in range(len(l)):
        dd = np.array_split(l[i],29,axis=1)
        input_data_split += dd
    input_data_split = np.array(input_data_split)
    return input_data_split

In [8]:
input_data_split = []
lat = 256
long = 256
for i in range(len(eta_t_numpy)):
    input_data = eta_t_numpy[i,:,:]
    print(np.array(input_data).shape)
    fix_missing_value(input_data)

    normalize_matrix(input_data)

#     input_data_split.append(split_array(input_data)[100])

(1500, 3600)
(1500, 3600)
(1500, 3600)
(1500, 3600)
(1500, 3600)
(1500, 3600)
(1500, 3600)
(1500, 3600)
(1500, 3600)
(1500, 3600)
(1500, 3600)
(1500, 3600)
(1500, 3600)
(1500, 3600)
(1500, 3600)
(1500, 3600)
(1500, 3600)
(1500, 3600)
(1500, 3600)
(1500, 3600)
(1500, 3600)
(1500, 3600)
(1500, 3600)
(1500, 3600)
(1500, 3600)
(1500, 3600)
(1500, 3600)
(1500, 3600)
(1500, 3600)
(1500, 3600)
(1500, 3600)
(1500, 3600)
(1500, 3600)
(1500, 3600)
(1500, 3600)
(1500, 3600)
(1500, 3600)
(1500, 3600)
(1500, 3600)
(1500, 3600)
(1500, 3600)
(1500, 3600)
(1500, 3600)
(1500, 3600)
(1500, 3600)
(1500, 3600)
(1500, 3600)
(1500, 3600)
(1500, 3600)
(1500, 3600)
(1500, 3600)
(1500, 3600)
(1500, 3600)
(1500, 3600)
(1500, 3600)
(1500, 3600)
(1500, 3600)
(1500, 3600)
(1500, 3600)
(1500, 3600)


In [ ]:
input_data_split= np.array(input_data_split)

In [ ]:
np.save('128x128_6months',input_data_split)

# split train set & validation set

In [ ]:
train_set, test_set = train_test_split(input_data_split, test_size=0.33)
train_set.shape

# Autoencoder

In [ ]:
lat = 1024
long = 1024

input_img = keras.Input(shape=(lat, long,1))

x = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(8, (3, 3), activation='relu',strides=(1,1))(x)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(4, (3, 3), activation='relu', padding='same',strides=(2,2))(x)
encoded = layers.MaxPooling2D((2, 2), padding='same')(x)

x = layers.Conv2DTranspose(8, (3, 3), activation='relu', padding='same')(encoded)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2DTranspose(8, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2DTranspose(16, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((4, 4))(x)
decoded = layers.Conv2DTranspose(1, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder = keras.Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='mse')
autoencoder.summary()

In [ ]:
history = autoencoder.fit(train_set, train_set,
                epochs=100, validation_data=(test_set, test_set))

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()